<a href="https://colab.research.google.com/github/iamsachinbagale/Transformer_Embeddings/blob/main/Text_Classification_using_Sentence_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -U sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 63.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
np.random.seed(2022)

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
df = pd.read_csv('/content/drive/MyDrive/toxic comment.csv',engine='python')

In [5]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
df = df[['comment_text', 'toxic']]

In [7]:
df.to_csv('filtered_file.csv', index=False)

In [8]:
df.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [9]:
import spacy
import string
nlp= spacy.load('en_core_web_sm')
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'amongst', 'without', 'former', 'although', 'there', '’s', 'whatever', 'which', 'his', 'himself', 'move', 'onto', 'to', 'empty', 'had', 'whence', 'same', 'she', 'fifteen', 'four', 'always', 'otherwise', 'fifty', 'top', 'everything', 'or', 'what', 'sometimes', 'along', 'from', 'herself', 'show', 'each', 'thru', 'take', 'am', 'besides', 'became', 'ourselves', 'both', 'in', 'say', 'forty', 'namely', 'its', 'already', "'m", 'i', 'many', "'ll", "n't", 'keep', '‘s', 'between', 'now', 'afterwards', 'until', '‘ll', 'via', 'much', 'perhaps', 'ours', 'others', "'d", 'side', 'seemed', 'him', 'eleven', 'within', 'five', 'least', 'ever', 'ca', 'an', 'ten', 'somehow', 'see', 'we', "'re", 'did', 'whenever', 'twelve', 'us', '’ve', 'than', 'after', 'indeed', 'me', 'this', 'too', 'hereby', 'two', 'nobody', 'why', 'would', 'about', 'should', 're', 'on', 'sixty', 'n’t', 'part', 'three', 'across', 'any', 'make', 'the', 'here', 'other', 'under', 'well', 'together', 'except', 'out', 'doing', 'whereas', 'the

In [10]:
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [11]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
  # Creating our token object, which is used to create doxuments with linguistic annotations
  doc = nlp(sentence)

  #print(doc)
  #print(type(doc))

  # Lemmatizing each token and converting each token into lower case
  mytokens = [ word.lemma_.lower().strip() for word in doc ]

  #print(my_tokens)

  # Removing stop words and punctuation
  mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

  sentence = " ".join(mytokens)
  # return preprocessed list of tokens
  return sentence

In [12]:
spacy_tokenizer("I am Happy!")

'happy'

In [14]:
df['tokenize'] = df['comment_text'].apply(spacy_tokenizer)

In [15]:
df['embeddings'] = df['tokenize'].apply(model.encode)

In [16]:
df.head()

,comment_text,toxic,tokenize,embeddings
0,Explanation\nWhy the edits made under my usern...,0,explanation edit username hardcore metallica f...,"[-0.030693008, 0.041847385, 0.04161458, 0.0137..."
1,D'aww! He matches this background colour I'm s...,0,d'aww match background colour seemingly stick ...,"[-0.0530621, 0.058449283, 0.044010296, -0.0059..."
2,"Hey man, I'm really not trying to edit war. It...",0,hey man try edit war guy constantly remove rel...,"[-0.021047903, 0.026711807, 0.003510148, 0.035..."
3,"""\nMore\nI can't make any real suggestions on ...",0,real suggestion improvement wonder section sta...,"[-0.047509678, -0.03601597, 0.042496487, 0.057..."
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page,"[-0.09303553, 0.008844864, -0.07133492, -0.007..."


In [18]:
X = df['embeddings'].to_list()
y = df['toxic'].to_list()

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [20]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [22]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.9521228262572458
Logistic Regression Precision: 0.8364835164835165
Logistic Regression Recall: 0.6220987250735535
